# Azure AI Search 구성 및 문서 인덱싱
이 스크립트는 Azure AI Search 서비스를 구성하고, 인덱스를 생성하며, 문서를 인덱싱합니다. 또한 특정 사용자를 위한 문서를 검색하는 함수를 정의합니다. 서비스 이름, 인덱스 이름, 키와 같은 민감한 정보는 보안을 위해 환경 변수에서 가져옵니다.

## 환경 변수
- `AZURE_SERVICE_NAME` : Azure Search 서비스의 이름.
- `AZURE_INDEX_NAME`: 인덱스의 이름.
- `AZURE_ADMIN_KEY`: Azure Search 서비스의 관리자 키.
- `AZURE_OPENAI_API_KEY`: Azure OpenAI의 API 키.
- `AZURE_OPENAI_RESOURCE_URI`: Azure OpenAI의 리소스 URI.
- `AZURE_OPENAI_DEPLOYMENT_ID`: Azure OpenAI의 배포 ID.
- `AZURE_OPENAI_MODEL_NAME`: Azure OpenAI의 모델 이름.

In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import json
import requests

# Load environment variables
service_name = os.getenv("AZURE_SERVICE_NAME")
index_name = os.getenv("AZURE_INDEX_NAME")
admin_key = os.getenv("AZURE_ADMIN_KEY")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_resource_uri = os.getenv("AZURE_OPENAI_RESOURCE_URI")
openai_deployment_id = os.getenv("AZURE_OPENAI_DEPLOYMENT_ID")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

# Payload Header 설정
headers = {
    'Content-Type': 'application/json',
    'api-key': admin_key
}
params = {'api-version': "2024-07-01"}

# Index Payload 설정
index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [
            {
                "name": "myalgo",
                "kind": "hnsw"
            }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters": {
                    "resourceUri": openai_resource_uri,
                    "apiKey": openai_api_key,
                    "deploymentId": openai_deployment_id,
                    "modelName": openai_model_name
                }
            }
        ],
        "profiles": [
            {
                "name": "myprofile",
                "algorithm": "myalgo",
                "vectorizer": "openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "subject"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "content"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": "true",
            "analyzer": "keyword",
            "searchable": "true",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "subject",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "facetable": "false",
            "filterable": "true",
            "sortable": "false"
        },
        {
            "name": "content",
            "type": "Edm.String",
            "searchable": "true",
            "retrievable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "allowed_groups",
            "type": "Collection(Edm.String)",
            "filterable": "true",
            "retrievable": "false"
        },
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536,
            "vectorSearchProfile": "myprofile",
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
    ]
}

# 인덱스 생성
r = requests.put(
    f"https://{service_name}.search.windows.net/indexes/{index_name}",
    data=json.dumps(index_payload),
    headers=headers,
    params=params
)
print(r.status_code)
print(r.ok)
print(r.text)

# 문서 데이터
documents = [
    {
        "id": "1",
        "subject": "신입 직원 온보딩 프로세스",
        "content": "신입 직원은 매주 월요일 오전 10시에 열리는 온보딩 세션에 참석해야 합니다. 이 세션에서는 회사 소개, 팀 미팅, 초기 서류 작업이 이루어집니다.",
        "allowed_groups": ["ITGroup","HRGroup"]
    },
    {
        "id": "2",
        "subject": "행동 강령",
        "content": "직원은 전문적인 태도를 유지하고, 동료를 존중하며, 회사의 핵심 가치를 준수해야 합니다.",
        "allowed_groups": ["HRGroup"]
    },
    {
        "id": "3",
        "subject": "원격 근무 가이드라인",
        "content": "원격 근무는 매니저의 승인을 받아야 하며, 역할이 허용되고 성과 기준을 충족해야 가능합니다.",
        "allowed_groups": ["ITGroup"]
    }
]

# 문서 인덱싱
index_documents_url = f"https://{service_name}.search.windows.net/indexes/{index_name}/docs/index"
index_documents_payload = {"value": documents}

r = requests.post(
    index_documents_url,
    data=json.dumps(index_documents_payload),
    headers=headers,
    params=params
)
print(r.status_code)
print(r.ok)
print(r.text)

# 사용자별 문서 검색 함수 정의
def search_documents_for_user(user_id):
    search_client = SearchClient(
        endpoint=f"https://{service_name}.search.windows.net",
        index_name=index_name,
        credential=AzureKeyCredential(admin_key)
    )

    # 사용자별 필터링 적용
    filter_query = f"allowed_groups/any(u:search.in(u, '{user_id}'))"
    results = search_client.search(search_text="*", filter=filter_query)

    for result in results:
        print(f"Document ID: {result['id']}, Content: {result['content']}")


204
True

200
True
{"@odata.context":"https://jhai-lab-search.search.windows.net/indexes('company-index')/$metadata#Collection(Microsoft.Azure.Search.V2024_07_01.IndexResult)","value":[{"key":"1","status":true,"errorMessage":null,"statusCode":200},{"key":"2","status":true,"errorMessage":null,"statusCode":200},{"key":"3","status":true,"errorMessage":null,"statusCode":200}]}


In [2]:
# 예제 실행: HRGroup이 접근할 수 있는 문서 검색
search_documents_for_user("HRGroup")

Document ID: 2, Content: 직원은 전문적인 태도를 유지하고, 동료를 존중하며, 회사의 핵심 가치를 준수해야 합니다.
Document ID: 1, Content: 신입 직원은 매주 월요일 오전 10시에 열리는 온보딩 세션에 참석해야 합니다. 이 세션에서는 회사 소개, 팀 미팅, 초기 서류 작업이 이루어집니다.


In [3]:
# 예제 실행: ITGroup이 접근할 수 있는 문서 검색
search_documents_for_user("ITGroup")

Document ID: 3, Content: 원격 근무는 매니저의 승인을 받아야 하며, 역할이 허용되고 성과 기준을 충족해야 가능합니다.
Document ID: 1, Content: 신입 직원은 매주 월요일 오전 10시에 열리는 온보딩 세션에 참석해야 합니다. 이 세션에서는 회사 소개, 팀 미팅, 초기 서류 작업이 이루어집니다.
